# Neural Networks

In [39]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.linear_model import (LinearRegression , LogisticRegression , Lasso)
from sklearn.preprocessing import StandardScaler 
from sklearn.model_selection import KFold
from sklearn.pipeline import Pipeline
from ISLP import load_data
from ISLP.models import ModelSpec as MS 
from sklearn.model_selection import (train_test_split , GridSearchCV)

from ISLP.models import (ModelSpec as MS, summarize , poly)
import statsmodels.api as sm

import torch.nn.functional as F #for one hot encoding  

import torch
from torch import nn
from torch.optim import RMSprop
from torch.utils.data import TensorDataset
from torchmetrics import (MeanAbsoluteError , R2Score, MeanSquaredError)
from torchinfo import summary
from torchvision.io import read_image

from pytorch_lightning import Trainer
from pytorch_lightning.loggers import CSVLogger
from pytorch_lightning import seed_everything
seed_everything(0, workers=True) 
torch.use_deterministic_algorithms(True, warn_only=True)

from ISLP.torch import (SimpleDataModule , SimpleModule , ErrorTracker , rec_num_workers)

from ISLP.torch.imdb import (load_lookup ,load_tensor , load_sparse ,load_sequential)

Seed set to 0


Load data and standardize

In [40]:
Df = pd.read_csv('Faellesdata_cleaned.csv')
X = Df.drop(columns = 'IsMigratorInt')
scaler = StandardScaler(with_mean=True, with_std=True)
#Scale all columns in X
X_scaled = scaler.fit_transform(X)


x_train, x_test, y_train, y_test = train_test_split(X_scaled, Df['IsMigratorInt'], test_size = 0.2, random_state = 42)

Prepare the tensors

In [42]:
x_train_t = torch.tensor(x_train.astype(np.float32))
y_train_t = torch.tensor(y_train.to_numpy().astype(np.float32))
train_set = TensorDataset(x_train_t , y_train_t)

x_test_t = torch.tensor(x_test.astype(np.float32))
y_test_t = torch.tensor(y_test.to_numpy().astype(np.float32))
test_set = TensorDataset(x_test_t , y_test_t)

Now lets set up our neural network.